In [125]:
import pandas as pd
import numpy as np
import random
courses= pd.ExcelFile("courses.xlsx")


In [126]:
courses.sheet_names[0]

'Robotics'

In [127]:
coursesDic=dict()
for sheetnum,sheet in enumerate(courses.sheet_names):
    
    # Parse data from each worksheet as a Pandas DataFrame and append it 
    # to the dic
    df = courses.parse(sheet)
    df.set_index('name',inplace=True)
    df["name_col"]=df.index
    coursesDic[courses.sheet_names[sheetnum]]=df

In [128]:
coursesDic2=coursesDic.copy()
coursesDic2["Fluid"].head()


,name_col
name,
ابرام عاطف جرجس كامل,ابرام عاطف جرجس كامل
احمد سعيد عبد الفتاح خليل مرسي,احمد سعيد عبد الفتاح خليل مرسي
احمد شعبان محمد احمد الفقي,احمد شعبان محمد احمد الفقي
احمد محمد محمد كامل,احمد محمد محمد كامل
احمد محمد مفتاح,احمد محمد مفتاح


In [129]:
students=pd.DataFrame(columns=['name'])
students.set_index('name',inplace=True)
students["e"]=0

for course in coursesDic2.values():

  students=pd.merge(students,course, how='outer',left_index=True, right_index=True).dropna(axis=1)
  

students.tail(50)

""
name
محمد السيد عبد المقصود السيد لادعم
محمد بدوي رمضان متولي شعبان
محمد جابر حمدي عبد القادر
محمد حسن محمد ابراهيم
محمد حسني محمد علي
محمد خالد محمد عبد الحميد مريز
محمد خالد محمد عبدالباقي
محمد رجب ابو سريع مغيب
محمد سيد عبد العليم عبد الواحد


In [130]:
def isInCourse(course):

  try:
    return coursesDic2[course].loc[students.index[118]].count()>0
  except:
    return False

studentMCourses=[]
for course in coursesDic2.keys():
  if isInCourse(course):
    studentMCourses.append(course)


In [131]:
studentMCourses

['Robotics',
 'NumericalControl',
 'AVS',
 'ManufSystemAutomation',
 'PLC',
 'SensorsAndActuators',
 'DigitalControl']

In [132]:
def MAD(x):
  '''
  calculates the median absolute deviation which is defined as median(|x_i - median(x)|
  inputs: arraylike 
          X
  outputs: numpy float
          MAD
  '''
  return np.median(np.absolute(x - np.median(x)))


In [133]:

'''
def isIntersected(course1,course2):
  
 # checks the intersection of students between two courses
 # inputs: str
 #         course 1 name , course 2 name
 # outputs: bool
 #          True if there is intersection and False otherwise
  
  df_merge = pd.merge(coursesDic[course1], coursesDic[course2], how='inner',left_index=True, right_index=True)
  return not len(df_merge)==0

'''



"\ndef isIntersected(course1,course2):\n  \n # checks the intersection of students between two courses\n # inputs: str\n #         course 1 name , course 2 name\n # outputs: bool\n #          True if there is intersection and False otherwise\n  \n  df_merge = pd.merge(coursesDic[course1], coursesDic[course2], how='inner',left_index=True, right_index=True)\n  return not len(df_merge)==0\n\n"

In [134]:
def isTaken(course):
  '''
  checks whether the courses is already in the time table or not
  inputs: str
          course name
  outputs: bool
           True if exists in the table, False otherwise
  '''
  t1=any(timeTable["course1"].astype(str).str.match(course))
  t2=any(timeTable["course2"].astype(str).str.match(course))
  t3=any(timeTable["course3"].astype(str).str.match(course))

  return t1 or t2 or t3

In [135]:
'''timeTablelist = {'course1':[], 'course2': []}
timeTable = pd.DataFrame(data=timeTablelist)
for i in range(10000):
  randCourseI=random.choice(list(coursesDic.keys()))
  randCourseJ=random.choice(list(coursesDic.keys()))

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):
    if(isIntersected(randCourseI,randCourseJ)==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)
'''

"timeTablelist = {'course1':[], 'course2': []}\ntimeTable = pd.DataFrame(data=timeTablelist)\nfor i in range(10000):\n  randCourseI=random.choice(list(coursesDic.keys()))\n  randCourseJ=random.choice(list(coursesDic.keys()))\n\n  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):\n    if(isIntersected(randCourseI,randCourseJ)==False):\n      \n\n      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ]],\n                                 columns=timeTable.columns)\n      timeTable=timeTable.append(dayNCourses,ignore_index=True)\n      coursesDic.pop(randCourseI)\n      coursesDic.pop(randCourseJ)\n"

In [136]:


def isIntersected(*args):
  '''
  checks the intersection of students between two courses
  inputs: str
          course 1 name , course 2 name
  outputs: bool
           True if there is intersection and False otherwise
  '''

  df_merge=pd.concat(list(args))
  isduplicated=df_merge.duplicated()
  return not len(df_merge[isduplicated])==0


timeTableDic = {'course1':[], 'course2': [],'course3':[]}
timeTable = pd.DataFrame(data=timeTableDic)

for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
    randCourseJ=random.choice(list(coursesDic.keys()))
    randCourseK=random.choice(list(coursesDic.keys()))
  except:
    None

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)and  not isTaken(randCourseK)):
    if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ],coursesDic2[randCourseK])==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,randCourseK]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)
      coursesDic.pop(randCourseK)



for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
    randCourseJ=random.choice(list(coursesDic.keys()))
  except:
    None

  if(not isTaken(randCourseI) and  not isTaken(randCourseJ)):
    if(isIntersected(coursesDic2[randCourseI],coursesDic2[randCourseJ])==False):
      

      dayNCourses = pd.DataFrame([[randCourseI,randCourseJ,"-"]],
                                 columns=timeTable.columns)
      timeTable=timeTable.append(dayNCourses,ignore_index=True)
      coursesDic.pop(randCourseI)
      coursesDic.pop(randCourseJ)


for i in range(10000):
  try:
    randCourseI=random.choice(list(coursesDic.keys()))
  except:
    None
    

  if(not isTaken(randCourseI)):


    coursesDic.pop(randCourseI)

    dayNCourses = pd.DataFrame([[randCourseI,"-","-"]],
                                columns=timeTable.columns)
    timeTable=timeTable.append(dayNCourses,ignore_index=True)


In [137]:
timeTable

,course1,course2,course3
0,Thermodynamics,Physics,PLC
1,calc1,AVS,Logic
2,LinearAlgebra,Mechatronics1,Electronics1
3,FundOfManuf,Signal,EngineeringDrawing
4,MechanicsAndThermodynamics,Robotics,MachineDesign2
5,Calc3,Mechanics1,ManufSystemAutomation
6,FundOfMangement,accounting,-
7,DigitalControl,CSAndProgramming,-
8,Control1,Microcontroller,-
9,SensorsAndActuators,Fields,-


In [138]:
coursesDic.keys()

dict_keys([])

In [139]:
#shuffling
timeTable=timeTable.sample(frac = 1,ignore_index=True)
timeTable

,course1,course2,course3
0,Calc3,Mechanics1,ManufSystemAutomation
1,FundOfMangement,accounting,-
2,InudstrialElectronics,-,-
3,NumericalControl,-,-
4,Fluid,-,-
5,SensorsAndActuators,Fields,-
6,DigitalControl,CSAndProgramming,-
7,LinearAlgebra,Mechatronics1,Electronics1
8,FundOfManuf,Signal,EngineeringDrawing
9,TechnicalWriting,-,-


In [148]:
studentMTT=timeTable[timeTable["course1"].astype(str).str.match("|".join(studentMCourses)) | 
                     timeTable["course2"].astype(str).str.match("|".join(studentMCourses)) |
                     timeTable["course3"].astype(str).str.match("|".join(studentMCourses))]
index=studentMTT.index
display(studentMTT)


,course1,course2,course3
0,Calc3,Mechanics1,ManufSystemAutomation
3,NumericalControl,-,-
5,SensorsAndActuators,Fields,-
6,DigitalControl,CSAndProgramming,-
10,calc1,AVS,Logic
12,MechanicsAndThermodynamics,Robotics,MachineDesign2
13,Thermodynamics,Physics,PLC


In [149]:
print(f"index MAD: {MAD(index)}")


index MAD: 4.0


In [143]:
MAD([0,5,1,0,1]*12)

1.0

In [144]:
MAD([1,3,4,30])

1.5

In [145]:
MAD([3,6,9,12])

3.0